**SQL Assignment 3**

In [1]:
-- use AdventureWorks data
USE AdventureWorks2014

Commands completed successfully.

Total execution time: 00:00:00.030

**Problem 1**

In [2]:
SELECT
    a.Name AS TerritoryName,
    CASE WHEN EXISTS (
            SELECT 1
            FROM Sales.SalesOrderHeader b
            WHERE b.TerritoryID = a.TerritoryID
            AND DATEPART(yy, b.OrderDate) = 2013
            GROUP BY DATEPART(mm, b.OrderDate)
            HAVING ROUND(SUM(b.SubTotal), 0) > 750000) 
        THEN 'No'
        ELSE 'Yes'
    END AS TrainingRequired
FROM Sales.SalesTerritory a
ORDER BY TerritoryName

(10 rows affected)

Total execution time: 00:00:00.105

TerritoryName,TrainingRequired
Australia,Yes
Canada,No
Central,Yes
France,No
Germany,Yes
Northeast,Yes
Northwest,No
Southeast,Yes
Southwest,No
United Kingdom,Yes


**Problem 2**

In [3]:
-- Part A

SELECT cr.Name as Country, sp.Name as StateProvince, COUNT(soh.SalesOrderID) AS TotalOrders
FROM Person.CountryRegion cr 
INNER JOIN Person.StateProvince sp 
ON cr.CountryRegionCode = sp.CountryRegionCode
INNER JOIN Person.Address a 
ON  sp.StateProvinceID = a.StateProvinceID
INNER JOIN Sales.SalesOrderHeader soh 
ON a.AddressID = soh.ShipToAddressID
GROUP BY ROLLUP(
    cr.Name,
    sp.Name)
HAVING cr.Name IS NOT NULL 
OR sp.Name IS NOT NULL
ORDER BY cr.Name, TotalOrders DESC

(76 rows affected)

Total execution time: 00:00:00.098

Country,StateProvince,TotalOrders
Australia,NULL,6843
Australia,New South Wales,3009
Australia,Victoria,1710
Australia,Queensland,1461
Australia,South Australia,466
Australia,Tasmania,197
Canada,NULL,4067
Canada,British Columbia,3472
Canada,Ontario,339
Canada,Quebec,146


In [4]:
-- Part B
-- I'm assuming you don't include the totals for each country 

SELECT cr.Name as Country, sp.Name as StateProvince, COUNT(soh.SalesOrderID) AS TotalOrders
FROM Person.CountryRegion cr 
INNER JOIN Person.StateProvince sp 
ON cr.CountryRegionCode = sp.CountryRegionCode
INNER JOIN Person.Address a 
ON  sp.StateProvinceID = a.StateProvinceID
INNER JOIN Sales.SalesOrderHeader soh 
ON a.AddressID = soh.ShipToAddressID
GROUP BY ROLLUP(
    cr.Name,
    sp.Name)
HAVING (cr.Name IS NOT NULL AND sp.Name IS NOT NULL)
    AND COUNT(soh.SalesOrderID) >= 100
ORDER BY cr.Name, TotalOrders DESC

(26 rows affected)

Total execution time: 00:00:00.129

Country,StateProvince,TotalOrders
Australia,New South Wales,3009
Australia,Victoria,1710
Australia,Queensland,1461
Australia,South Australia,466
Australia,Tasmania,197
Canada,British Columbia,3472
Canada,Ontario,339
Canada,Quebec,146
France,Seine (Paris),575
France,Seine Saint Denis,391


**Problem 3**

In [5]:
SELECT 
    p.Name as Product, 
    pc.Name as Category, 
    SUM(sod.OrderQty) as TotalUnitsSold,
    RANK() OVER (PARTITION BY pc.Name ORDER BY SUM(sod.OrderQty) DESC) AS Rank,
    NTILE(5) OVER (PARTITION BY pc.Name ORDER BY SUM(sod.OrderQty) DESC) AS Quintile
FROM Production.ProductCategory pc 
INNER JOIN Production.ProductSubcategory ps 
ON pc.ProductCategoryID = ps.ProductCategoryID
INNER JOIN Production.Product p 
ON ps.ProductSubcategoryID = p.ProductSubcategoryID
INNER JOIN Sales.SalesOrderDetail sod
ON p.ProductID = sod.ProductID
GROUP BY 
    p.Name,
    pc.Name 
ORDER BY Category, TotalUnitsSold DESC

(266 rows affected)

Total execution time: 00:00:00.194

Product,Category,TotalUnitsSold,Rank,Quintile
Water Bottle - 30 oz.,Accessories,6815,1,1
"Sport-100 Helmet, Blue",Accessories,6743,2,1
"Sport-100 Helmet, Black",Accessories,6532,3,1
"Sport-100 Helmet, Red",Accessories,6266,4,1
Patch Kit/8 Patches,Accessories,3865,5,1
Bike Wash - Dissolver,Accessories,3319,6,2
Hitch Rack - 4-Bike,Accessories,3166,7,2
Mountain Tire Tube,Accessories,3095,8,2
Hydration Pack - 70 oz.,Accessories,2761,9,2
Road Tire Tube,Accessories,2376,10,2
